In [2]:
import geemap
import os
import ee
geemap.set_proxy(port=7890)

In [3]:
Map=geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [4]:
geemap.ee_search()

<IPython.core.display.Javascript object>

In [9]:
import ee 
import geemap

# Create a map centered at (lat, lon).
Map = geemap.Map(center=[40, -100], zoom=4)

# This example uses the Sentinel-2 QA band to cloud mask
# the collection.  The Sentinel-2 cloud flags are less
# selective, so the collection is also pre-filtered by the
# CLOUDY_PIXEL_PERCENTAGE flag, to use only relatively
# cloud-free granule.

# Function to mask clouds using the Sentinel-2 QA band.
# function maskS2clouds(image) {
def maskS2clouds(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))

  # Return the masked and scaled data, without the QA bands.
  return image.updateMask(mask).divide(10000) \
      .select("B.*") \
      .copyProperties(image, ["system:time_start"])
# }

# Map the function over one year of data and take the median.
# Load Sentinel-2 TOA reflectance data.
collection = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate('2016-09-01', '2016-10-30') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .map(maskS2clouds)

composite = collection.median().clip(roi)

# Display the results.
Map.addLayer(composite, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}, 'RGB')

# Display the map.
Map


Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [6]:
roi= ee.FeatureCollection('users/cugbxwd/qaidam').geometry()
Map.addLayer(roi, {}, "qaidam")

In [10]:
def maskL8sr(image):
  # Bits 3 and 5 are cloud shadow and cloud, respectively.
  cloudShadowBitMask = 1 << 3
  cloudsBitMask = 1 << 5

  # Get the pixel QA band.
  qa = image.select('pixel_qa')

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
      .And(qa.bitwiseAnd(cloudsBitMask).eq(0))

  # Return the masked image, scaled to reflectance, without the QA bands.
  return image.updateMask(mask).divide(10000) \
      .select("B[0-9]*") \
      .copyProperties(image, ["system:time_start"])
# }

In [12]:
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterDate('2013-07-01', '2013-9-30')\
    .map(maskL8sr)

composite = collection.select(['B4', 'B3', 'B2']).median().clip(roi)

# Display the results.
Map.addLayer(composite, { 'min': 0, 'max': 0.3}, 'Image')

# Display the map.
Map

Map(bottom=25784.0, center=[37.10776507118514, -265.8773803710938], controls=(WidgetControl(options=['position…

In [ ]:
Map.draw_features

In [ ]:
roi = ee.FeatureCollection(Map.draw_features).geometry()

In [ ]:
print(composite.getInfo())

In [13]:
geemap.ee_export_image_to_drive(composite, "qaidam_landsat", folder="Qaidam", region=roi , scale=300, crs=None, file_format='GeoTIFF',format_options={ 'cloudOptimized': True})

Exporting qaidam_landsat ...


In [ ]:
roi = ee.FeatureCollection('users/cugbxwd/resize').geometry()
Map.addLayer(roi, {}, "resize")
Map.centerObject(roi, 7)

In [ ]:
geemap.ee_search()

In [ ]:
images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate('2015-01-01', '2019-12-31') \
    .sort('CLOUD_COVER')

In [ ]:
print(images.size().getInfo())

In [ ]:
best_image = images.first().select(['B1', 'B2', 'B3', 'B4',  'B5', 'B6', 'B7'])

In [ ]:
print(best_image.getInfo())

In [ ]:
dataset_jxd = ee.ImageCollection("MODIS/006/MOD09Q1")
falseColorVis = {
  'min': -100.0,
  'max': 8000.0,
  'bands': ['sur_refl_b02', 'sur_refl_b02', 'sur_refl_b01'],
}
Map.addLayer(dataset_jxd, falseColorVis, "MODIS/006/MOD09Q1")

In [ ]:
collections=dataset_jxd.filterBounds(roi) .filterDate('2020-01-01', '2021-01-01').median().clip(roi)
falseColorVis = {
  'min': -100.0,
  'max': 8000.0,
  'bands': ['sur_refl_b02', 'sur_refl_b02', 'sur_refl_b01'],
}
Map.addLayer(collections, falseColorVis, "coleections")

In [ ]:
collections.size().getInfo()

In [ ]:
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(roi) \
    .filterDate('2020-01-01', '2021-01-01')

In [ ]:
image = collection.median().clipToCollection(roi)
Map.addLayer(image, {'bands': ['B3', 'B2', 'B5'], 'max': 0.3}, 'Landsat 2020')

In [ ]:
list=[]


In [ ]:
images = geemap.landsat_timeseries(roi=roi,start_year=1990, end_year=2020, start_date='01-01', end_date='12-31')

In [ ]:
first = images.first()

vis_params = {
    'bands': ['NIR', 'Red', 'Green'],
    'min': 0,
    'max': 3000
}

Map.addLayer(first, vis_params, 'First image')

In [ ]:
ndwi_images = images.map(lambda img: img.normalizedDifference(['Green', 'SWIR1']).rename('ndwi'))

ndwi_palette = ['#ece7f2', '#d0d1e6', '#a6bddb', '#74a9cf', '#3690c0', '#0570b0', '#045a8d', '#023858']

first_ndwi = ndwi_images.first()

Map.addLayer(first_ndwi, {'palette': ndwi_palette}, 'First NDWI')

In [ ]:
water_images = ndwi_images.map(lambda img: img.gt(0).selfMask())

first_water = water_images.first()

Map.addLayer(first_water, {'palette': ['blue']}, 'First Water')

In [ ]:
list=[]
for i in range(31):
    a=1990
    b=a+i
    list.append('water'+str(b))
print(list)
  
    

In [ ]:
def cal_area(img):
    pixel_area = img.multiply(ee.Image.pixelArea()).divide(1e6)
    img_area = pixel_area.reduceRegion(**{
        'geometry': roi.geometry(),
        'reducer': ee.Reducer.sum(),
        'scale': 1000,
        'maxPixels': 1e12
    })
    return img.set({'water_area': img_area})

In [ ]:
water_areas = water_images.map(cal_area)

In [ ]:
water_stats = water_areas.aggregate_array('water_area').getInfo()
water_stats

In [ ]:
geemap.ee_export_image_collection_to_drive(water_images, descriptions=list, folder="ndwi_sjy", region=roi, scale=30, crs=None, file_format='GeoTIFF')

In [ ]:
print(best_image.propertyNames().getInfo())

In [ ]:
print(best_image.get('system:id').getInfo())

In [ ]:
image = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_132036_20170911')
vis_params = {
  'bands': ['B6', 'B5', 'B4'],
  'min': 0,
  'max': 6000,
  'gamma': 1.4,
}
Map.addLayer(image, vis_params, 'image')
Map